In [1]:
!pip install transformers datasets torch sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (

In [2]:
import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

In [17]:
dataset = load_dataset("wmt14", "de-en", split='train[:1%]')

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["de"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

def convert_to_dict(dataset):
    input_ids = []
    attention_masks = []
    labels = []

    for item in dataset:
        input_ids.append(item["input_ids"])
        attention_masks.append(item["attention_mask"])
        labels.append(item["labels"])

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

train_dataset = convert_to_dict(tokenized_datasets)

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/45088 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
class TranslationDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = TranslationDataset(train_dataset)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)  # Reduced batch size

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de").to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [20]:
model.train()
for epoch in range(1):  # Reduced to 1 epoch
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Streaming output truncated to the last 5000 lines.
Epoch: 0, Loss: 0.44900694489479065
Epoch: 0, Loss: 0.5092845559120178
Epoch: 0, Loss: 0.3686891198158264
Epoch: 0, Loss: 0.6371797323226929
Epoch: 0, Loss: 0.5913200378417969
Epoch: 0, Loss: 0.39326030015945435
Epoch: 0, Loss: 0.5529458522796631
Epoch: 0, Loss: 0.42986345291137695
Epoch: 0, Loss: 0.4428233504295349
Epoch: 0, Loss: 0.3832688331604004
Epoch: 0, Loss: 0.38824158906936646
Epoch: 0, Loss: 0.692170262336731
Epoch: 0, Loss: 0.35354992747306824
Epoch: 0, Loss: 0.33731746673583984
Epoch: 0, Loss: 0.3543768525123596
Epoch: 0, Loss: 0.26801326870918274
Epoch: 0, Loss: 0.3873922526836395
Epoch: 0, Loss: 0.4275333285331726
Epoch: 0, Loss: 0.40572410821914673
Epoch: 0, Loss: 0.30673786997795105
Epoch: 0, Loss: 0.34807780385017395
Epoch: 0, Loss: 0.3937971293926239
Epoch: 0, Loss: 0.2714461088180542
Epoch: 0, Loss: 0.4096754789352417
Epoch: 0, Loss: 0.3350929915904999
Epoch: 0, Loss: 0.3242941200733185
Epoch: 0, Loss: 0.492785722017

In [21]:
bleu = load_metric("sacrebleu")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
test_dataset = load_dataset("wmt14", "de-en", split='test[:1%]')

tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

def convert_to_list(dataset):
    data = []
    for item in dataset:
        data.append({
            "input_ids": item["input_ids"],
            "attention_mask": item["attention_mask"],
            "labels": item["labels"],
            "en": item["translation"]["en"],
            "de": item["translation"]["de"]
        })
    return data

test_data = convert_to_list(tokenized_test_dataset)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [23]:
def evaluate_model(model, tokenizer, dataset):
    model.eval()
    predictions, references = [], []
    for item in dataset:
        inputs = tokenizer(item["en"], return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            translated_tokens = model.generate(**inputs)
        decoded_preds = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        decoded_labels = [item["de"]]
        predictions.extend(decoded_preds)
        references.extend([[label] for label in decoded_labels])
    result = bleu.compute(predictions=predictions, references=references)
    return result

bleu_score = evaluate_model(model, tokenizer, test_data)
print(f"BLEU Score: {bleu_score['score']}")

BLEU Score: 16.500385046619932


In [24]:
def error_analysis(predictions, references):
    errors = []
    for pred, ref in zip(predictions, references):
        if pred != ref[0]:
            errors.append((pred, ref[0]))
    return errors

errors = error_analysis(predictions, references)
for error in errors[:5]:  # Print first 5 errors
    print(f"Predicted: {error[0]}")
    print(f"Reference: {error[1]}")

NameError: name 'predictions' is not defined

In [25]:
def sensitivity_analysis(model, tokenizer, dataset, seeds=[42, 43, 44]):
    results = []
    for seed in seeds:
        torch.manual_seed(seed)
        result = evaluate_model(model, tokenizer, dataset)
        results.append(result['score'])
    return results

sensitivity_results = sensitivity_analysis(model, tokenizer, test_data)
print(f"Sensitivity Analysis BLEU Scores: {sensitivity_results}")

Sensitivity Analysis BLEU Scores: [16.500385046619932, 16.500385046619932, 16.500385046619932]
